   
- Import dependencies
- creat s3 bucket with UI
- map train/test data to s3
- map the path of the models in s3
- xgboost
- deployment
- simple predictions

In [20]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
     

In [9]:
# set an output path where the trained model will be saved

prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format('bucket8912', prefix)
print(output_path) #model will be saves to that path

s3://bucket8912/xgboost-as-a-built-in-algo/output


In [10]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [11]:
## Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [13]:
model_data.head(2)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [15]:
# Saving Train Into Buckets
import os
#dependent features should be first column
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [17]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [21]:
# pulls XGBoost image URI and builds an XGBoost container.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [22]:
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [24]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=1, # 1 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,#reduce billing time
                                          train_max_run=300,#reduce billing time
                                          train_max_wait=600#reduce billing time)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [25]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-08-20-21-04-02-839


2023-08-20 21:04:03 Starting - Starting the training job...
2023-08-20 21:04:17 Starting - Preparing the instances for training.........
2023-08-20 21:05:46 Downloading - Downloading input data...
2023-08-20 21:06:27 Training - Training image download completed. Training in progress....
2023-08-20 21:06:57 Uploading - Uploading generated training model[2023-08-20 21:06:50.603 ip-10-0-75-84.ca-central-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
IN

In [36]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge',serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-08-20-21-35-02-637
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-08-20-21-35-02-637
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-08-20-21-35-02-637


------!

In [42]:
#csv has to be first serialized then passed to model
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values
xgb_predictor.content_type = 'text/csv' 
xgb_predictor.serializer =sagemaker.serializers.CSVSerializer()
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict

predictions_array = np.fromstring(predictions[1:], sep=',') # turn the prediction into an array
print(predictions_array.shape)  

(12357,)


In [43]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [44]:
#confusion matrix
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [45]:
#delete endpoint
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket('bucket8912')
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-08-20-21-35-02-637


[{'ResponseMetadata': {'RequestId': 'TS2NS802H9HBF87D',
   'HostId': 'HLY6FPBUP/RmvEm7jMazCGRz6Ffs1V+YVieLCQnADsEttZWmwctIOFSYM5TSv1RQxaTMwFKZ9J8=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'HLY6FPBUP/RmvEm7jMazCGRz6Ffs1V+YVieLCQnADsEttZWmwctIOFSYM5TSv1RQxaTMwFKZ9J8=',
    'x-amz-request-id': 'TS2NS802H9HBF87D',
    'date': 'Sun, 20 Aug 2023 21:45:36 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/test/test.csv',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'zFgy7lS4Uu0pb7VuW6pe5QWBD2dsoteX'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-08-20-21-04-02-839/debug-output/index/000000000/000000000040_worker_0.json',
    'DeleteMarker': True,
    'DeleteMarkerVersionId': 'EykpAx0iPlmDSupGq_vuw7N3Mo.YlY1c'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-08-20